<h1><center>Προχωρημένα Θέματα Βάσεων Δεδομένων</center></h1>
<h2><center>Εξαμηνιαία Εργασία</center></h2>
<hr>

| Ονοματεπώνυμο      | Αριθμός Μητρώου |
| ------------------ | --------------- |
| Δοντάς Σπυρίδων    | 03114141        |
| Τσιούρβας Αστέριος | 03114133        |

<hr>
<h1><center>Προεργασία</center></h1>

In [1]:
import datetime as dt
import pandas as pd

from IPython.display import display, HTML

### Διάβασμα δεδομένων σε `RDDs`

In [2]:
vendors = sc.textFile("/Project/yellow_tripvendors_1m.csv")
data = sc.textFile("/Project/yellow_tripdata_1m.csv")

<hr>
<h1><center>Ερώτημα 1</center></h1>

Σε αυτό το ερώτημα κληθήκαμε να υπολογίσουμε τα εξής:
* το μέσο διάστημα διαδρομής ανά ώρα εκκίνησης διαδρομής
* το μέγιστο ποσό που πληρώθηκε για κάθε εταιρεία

### Συνάρτηση για δημιουργία `key - value`

In [3]:
def start_duration(line):
    """Function to map over RDD lines.
    
    Produces the necessary tuples for key - value processing.
    The starttime of the trip becomes the key, while the duration
    of the trip becomes the value.
    
    Parameters
    ----------
    
    line: string
        CSV delimited string from RDD.
        
    Returns
    -------
    
    keyval: tuple of (string, float)
        Key   -> starttime
        Value -> duration
    """
    contents = line.split(",")
    start, end = contents[1], contents[2]
    starttime = dt.datetime.strptime(start, "%Y-%m-%d %H:%M:%S")
    endtime = dt.datetime.strptime(end, "%Y-%m-%d %H:%M:%S")
    diff = endtime - starttime
    diff = diff.total_seconds() / 60
    return "{:02d}".format(starttime.hour), diff

### Map
Με τη μέθοδο `map` δημιουργούμε ένα νέο `RDD` από `tuples` τις οποίες το `Spark` αντιμετωπίζει ως ζευγάρι `key - value`. Το παραπάνω θα ισχύει για όλες τις επόμενες χρήσεις μιας αρχικής κλήσης `map` πάνω στα αρχικά δεδομένα.
***
Συγκεκριμένα, εδώ θα έχουμε ως `key` την ώρα εκκίνησης της διαδρομής και ως `value` τη διάρκεια της διαδρομής. 

In [4]:
duration = data.map(start_duration)

### Εύρεση μέσης διάρκειας διαδρομής ανά ώρα εκκίνησης
* μετατρέπουμε κάθε τιμή σε `tuple` της μορφής `(value, 1)`
* εφαρμόζουμε `reduceByKey` με μια συνάρτηση η οποία αθροίζει τα στοιχεία κάθε τούπλας
  * σαν αποτέλεσμα για κάθε κλειδί θα έχουμε μία `tuple` της μορφής `(sum(values), num(values))`
* εφαρμόζουμε για κάθε τιμή (που είναι της μορφής `(sum, len)`) τη συνάρτηση `sum / num` (η οποία παράγει το μέσο όρο)

In [5]:
duration = duration.mapValues(lambda x: (x, 1))\
                   .reduceByKey(
                        lambda a, b: tuple(map(sum, zip(a, b)))
                    )\
                   .mapValues(lambda x: x[0] / x[1])

### Αποθήκευση αρχείου στο `HDFS`
* μετατρέπουμε το `RDD` σε `PySpark DataFrame` με `headers`
* μαζεύουμε σε ένα `partition`
* αποθηκεύουμε ως `csv`

In [6]:
df = duration.toDF(["HourOfDay", "AverageTripDuration"])\
             .sort("HourOfDay")
df.coalesce(1)\
  .write\
  .csv("/Project/TripDuration", header="true", mode="overwrite")

### Εμφάνιση αποτελεσμάτων

In [7]:
display(HTML(df.toPandas().to_html(index=False)))

HourOfDay,AverageTripDuration
00,14.017794
01,13.975070
02,13.035636
03,13.322283
04,13.799858
05,13.275583
06,12.487420
07,13.395006
08,14.627505
09,14.670106


### Συνάρτηση για δημιουργία `key - value`

In [8]:
def id_price(line):
    """Function to map over RDD lines.
    
    Produces the necessary tuples for key - value processing.
    The id of the trip becomes the key, while the price of
    the trip becomes the value.
    
    Parameters
    ----------
    
    line: string
        CSV delimited string from RDD.
        
    Returns
    -------
    
    idprice: tuple of (string, float)
        Key   -> id of trip
        Value -> price of trip
    """
    contents = line.split(",")
    return contents[0], float(contents[7])

### Map
Εφαρμόζουμε `map` τόσο στο `vendors RDD` όσο και στο `data RDD` ως εξής:
* για το `vendors` δημιουργούμε ένα νέο `RDD` με `key` το id της διαδρομής και `value` το id της εταιρείας
* για το `data` δημιουργούμε ένα νέο `RDD` με `key` το id της διαδρομής και `value` το ποσό που πληρώθηκε για τη διαδρομή

In [9]:
vendors = vendors.map(lambda x: tuple(x.split(",")))
prices = data.map(id_price)

### Inner Join
Ενώνουμε τα δύο `RDDs` σε ένα, εφαρμόζοντας inner join πάνω στο id της διαδρομής. Το παραγόμενο `RDD` αποτελείται από ειδική δομή του `PySpark` ονομαζόμενη `Row` και μοιάζει με το αντίστοιχο `dictionary` της `Python`.

**Σημείωση**: Χρειάστηκε να μετατρέψουμε τα `RDDs` σε `PySpark DataFrames` για τη βελτιστοποίηση του join.

In [10]:
vendors = vendors.toDF(["id", "vendor"])
prices = prices.toDF(["id", "price"])
prices = vendors.join(prices, on="id", how="inner")
prices = prices.rdd

### Εύρεση μέγιστης τιμής ανά εταιρεία
* εφαρμόζουμε `map` στο `RDD` για δημιουργία `key - value` με `key` το id της εταιρείας και `value` την τιμή της διαδρομής (το id της διαδρομής πλέον δε μας ενδιαφέρει)
* εφαρμόζουμε `reduceByKey` με τη συνάρτηση `max` για να βρούμε το μέγιστο για κάθε εταιρεία

In [11]:
prices = prices.map(lambda x: (x["vendor"], x["price"]))\
               .reduceByKey(max)

### Αποθήκευση αρχείου στο `HDFS`
* μετατρέπουμε το `RDD` σε `PySpark DataFrame` με `headers`
* μαζεύουμε σε ένα `partition`
* αποθηκεύουμε ως `csv`

In [12]:
df = prices.toDF(["VendorID", "MaxAmountPaid"])\
           .sort("VendorID")
df.coalesce(1)\
  .write\
  .csv("/Project/MaxAmountPaid", header="true", mode="overwrite")

### Εμφάνιση αποτελεσμάτων

In [13]:
display(HTML(df.toPandas().to_html(index=False)))

VendorID,MaxAmountPaid
1,503326.33
2,548463.35


<hr>
<h1><center>Ερώτημα 2</center></h1>

Σε αυτό το ερώτημα θα πρέπει εφαρμόζοντας τον αλγόριθμο `k-Means` να βρούμε τα κέντρα των top 5 περιοχών επιβίβασης.

### Ορισμός ευκλείδιας απόστασης

In [14]:
def euclidean(p, q):
    """Calculate euclidean distance.
    
    Formula:
    .. math::
        \sqrt{\sum (p_i - q_i)^2}
        
    Parameters
    ----------
    
    p: iterable of numbers
        Contains the coordinates of the first point.
        
    q: iterable of numbers
        Contains the coordinates of the second point.
        
    Returns
    -------
    
    euclidean: float
    """
    sum = 0
    for pi, qi in zip(p, q):
        sum += (pi - qi) ** 2
    return sum ** (1 / 2)

### Συνάρτηση για δημιουργία συντεταγμένων εκκίνησης

In [15]:
def coordinates(line):
    """Function to map over RDD lines.
    
    Produces the necessary Coordinates tuples.
    
    Parameters
    ----------
    
    line: string
        CSV delimited string from RDD.
        
    Returns
    -------
    
    coordinates: tuple of numbers
    """
    contents = line.split(",")
    lng, lat = map(float, contents[3:5])
    return lng, lat

### Αρχικοποιήσεις
* Εφαρμόζουμε `map` στο `data RDD` δημιουργώντας το `RDD population` αποτελούμενο από συντεταγμένες εκκίνησης σε `tuple`.
* `k = 5`, ο αριθμός των top κέντρων που θέλουμε να βρούμε.
* `iterations = 3`, ο αλγόριθμος θα κάνει 3 επαναλήψεις.
* Βρίσκουμε την αρχική περίπτωση `centroids`, τα οποία θεωρούμε ότι είναι τα πρώτα 5 στο `RDD population`.

In [16]:
population = data.map(coordinates)
k = 5
iterations = 3
centroids = [(i, c) for i, c in enumerate(population.take(k), 1)]

Η παραπάνω αρχικοποίηση των `centroids` θα μπορούσε να είχε γραφεί:
```python
centroids = list(enumerate(population.take(k), 1))
```

### Συνάρτηση για εύρεση κοντινότερου κέντρου

In [17]:
def closest(coords_centroids):
    """Find closest centroid.
    
    Computes the minimum euclidean distance
    and saves centroid id.
    
    Parameters
    ----------
    
    coords_centroids: tuple of tuple, iterable
        Tuple contains the tuple of coordinates, as well
        as the centroids as an iterable, with each centroid
        containing an id and a coordinate.
        
    Returns
    -------
    
    id_coords: tuple of int, tuple
        Tuple containing the id of the closest centroid and
        the tuple of coordinates of this point.
    """
    coordinates, centroids = coords_centroids
    distance = min(
        (
            (centroid[0], euclidean(coordinates, centroid[1])) 
            for centroid in centroids
        ),
        key=lambda x: x[1]
    )
    return (distance[0], coordinates)

### Συνάρτηση για άθροισμα εμφολευμένων `iterable`

In [18]:
def sum_by_elem(p, q):
    """Function to reduce over RDD values.
    
    Gets two elements and sums each coordinate.
    
    Parameters
    ----------
    
    p: tuple of tuple, int
        Contains tuple of floats and an int
    
    q: tuple of tuple, int
        Contains tuple of floats and an int
    
    Returns
    -------
    
    sum_by_elem: tuple of tuple, int
        Tuple of floats, int
    """
    p, num1 = p
    q, num2 = q
    return (tuple(map(sum, zip(p, q))), num1 + num2)

### Συνάρτηση για εύρεση μέσου όρου ανά στοιχείο

In [19]:
def avg_by_elem(p):
    """Function to reduce over RDD values.
    
    Gets an element and averages each coordinate.
    
    Parameters
    ----------
    
    p: tuple of tuple, int
        Contains a tuple with summed by element values, sum of ints
    
    Returns
    -------
    
    avg_by_elem: tuple of floats
        For each tuple element divide with number of elements
    """
    p, num = p
    return tuple(map(lambda x: x / num, p))

### Κύριος Αλγόριθμος
Σε κάθε iteration του αλγορίθμου ακολουθούμε τα εξής βήματα:
* εφαρμόζουμε `map` για να δημιουργήσουμε ένα `RDD` με τις συντεταγμένες κάθε σημείου και ένα αντίγραφο των κέντρων
* εφαρμόζουμε `map` με τη συνάρτηση `closest` στο παραπάνω `RDD` για να δημιουργήσουμε ένα `RDD` με `key` το id του κοντινότερου cluster και `value` τις συντεταγμένες του σημείου
* εφαρμόζουμε `mapValues`-`reduceByKey`-`mapValues` για την εύρεση των νέων κέντρων με βάση τους μέσους όρους ανά συντεταγμένη (η διαδικασία είναι παρόμοια με την εύρεση του μέσου όρου στο ερώτημα 1 με τη διαφορά ότι εδώ βρίσκουμε μέσο όρο σε πολλά στοιχεία ανά στοιχείο)

In [20]:
for _ in range(iterations):
    population_centroids = population.map(lambda x: (x, centroids))
    points_labels = population_centroids.map(closest)
    new_centroids = \
        points_labels.mapValues(lambda x: (x, 1))\
                     .reduceByKey(sum_by_elem)\
                     .mapValues(avg_by_elem)
    centroids = new_centroids.collect()

### Αποθήκευση αρχείου στο `HDFS`
* μετατρέπουμε το `RDD` σε `PySpark DataFrame` με `headers`
* μαζεύουμε σε ένα `partition`
* αποθηκεύουμε ως `csv`

In [21]:
centroids = new_centroids.toDF(["Id", "Centroid"])
centroids.withColumn("Centroid", centroids.Centroid.cast("string"))\
         .coalesce(1)\
         .write\
         .csv(
              "/Project/Centroids",
              header="true",
              mode="overwrite",
              quote=""
          )

### Εμφάνιση αποτελεσμάτων

In [23]:
display(HTML(centroids.toPandas().to_html(index=False)))

Id,Centroid
1,"(-78.5038663350961, 40.61111272363974)"
2,"(-0.000130403195932698, 0.0005087396653170571)"
3,"(-73.95393670658325, 40.69903130538949)"
4,"(-73.9921679218484, 40.742681085334965)"
5,"(-73.96113225987052, 40.77182523315749)"
